In [20]:
import csv
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle


In [21]:
DS = pd.read_csv("MergedDS.csv")

In [3]:
DS.text.apply(lambda x: len(x)).describe()

count    185888.000000
mean       2267.345504
std        4234.895677
min           4.000000
25%         431.000000
50%        1058.000000
75%        2443.000000
max      102602.000000
Name: text, dtype: float64

In [4]:
from collections import Counter

unique_symbols = Counter()

for _, message in DS.text.iteritems():
    unique_symbols.update(message)
    
print("Unique symbols:", len(unique_symbols))

Unique symbols: 253


In [5]:
THRESHOLD = 5
uncommon_symbols = list()

for symbol, count in unique_symbols.items():
    if count < THRESHOLD:
        uncommon_symbols.append(symbol)

print("Uncommon symbols:", len(uncommon_symbols))


Uncommon symbols: 30


In [6]:

# Replace them with a placeholder:

DUMMY = uncommon_symbols[0]
tr_table = str.maketrans("".join(uncommon_symbols), DUMMY * len(uncommon_symbols))

DS.text = DS.text.apply(lambda x: x.translate(tr_table))




In [9]:
# We will need the number of unique symbols further down when we will decide on the dimensionality of inputs.

num_unique_symbols = len(unique_symbols) - len(uncommon_symbols) + 1 

tokenizer = Tokenizer(
    char_level=True,
    filters=None,
    lower=False,
    num_words=num_unique_symbols
)

tokenizer.fit_on_texts(DS.text)
sequences = tokenizer.texts_to_sequences(DS.text)


In [11]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Pad the input: I use the 500 lenght, just a bit over the median length.

padded_sequences = pad_sequences(sequences, maxlen=500)


In [22]:
DS.text

0                                           RR 130/80 mmHg.
1         Kergelt liigkaaluline. Nahk, limaskestad taval...
2         RR142/89mmHg, HR 76 x min, p167cm, k89kg.  EKG...
3                   RR143/87mmHg, HR60 x min, p180cm, k93kg
4         RR 140/74 mm Hg Cor fr. 62 UH uuringul kaela a...
                                ...                        
185883    14.11.2014 -     Saabus kiirabiga. Kaebusteks ...
185884    *22.10.2014 <ANONYM id="0" type="per" morph="_...
185885    *03.11.2014 <ANONYM id="0" type="per" morph="_...
185886    *18.11.2014 <ANONYM id="0" type="per" morph="_...
185887     Anamnees 08.08.2014:\nPalavaga veidi raskem h...
Name: text, Length: 185888, dtype: object

In [18]:
padded_sequences

array([[ 0,  0,  0, ..., 49, 22,  7],
       [ 0,  0,  0, ...,  3,  5,  7],
       [ 0,  0,  0, ...,  4, 16,  7],
       ...,
       [ 0,  0,  0, ..., 41, 24, 24],
       [10, 12,  1, ..., 10,  4,  1],
       [ 0,  0,  0, ...,  1,  1, 19]])

In [15]:
DS.text = pad_sequences

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(DS, y_train, stratify=y_train['toxic'], test_size=0.05)
